# 載入需要套件

In [1]:
import os
import argparse
import pandas as pd
import numpy as np
import scipy.sparse as sp
from lightfm import LightFM
from lightfm.data import Dataset
from evaluation import Evaluation
from mlaas_tools.config_build import config_set
from db_connection.utils import get_conn
from utils import recommendation_all, load_w103, load_w106, load_cust_pop, create_all_feature_pairs, build_feature_tuples, load_cust_pop_0205 

In [2]:
## Configure env
if not os.path.isfile('config.ini'):
    config_set()

In [3]:
## Load db connection
rawdata_conn = get_conn('edu')

read key file
login as edu-agb


In [6]:
## Load data
today = '2018-12-31'
span = 18
print("Loading Data...")
w103_df = load_w103(today, rawdata_conn, span)

Loading Data...


In [7]:
purchase_hist = w103_df.groupby("cust_no")["wm_prod_code"].apply(lambda x: list(set(x.values.tolist()))).to_dict()

In [ ]:
warm_user = list(purchase_hist.keys() & self.ans.keys())
cold_user = list(ans.keys() - self.purchase_hist.keys())

In [ ]:
def answer(self, date, conn, duration):
        df = self.read(date, conn, duration)
        return df.groupby('cust_no')['wm_prod_code'].apply(list).to_dict()

    def read(self, date, conn, duration):
        after_1d_dt, after_1m_dt, after_7d_dt = self.get_data_dt(date, 1)

        if duration == '7d':
            d_end = after_7d_dt
        else:
            d_end = after_1m_dt

        sql = """
            with 
                cte1 as (select cust_id as cust_no, 
                                replace(wm_prod_code, ' ', '') as wm_prod_code, 
                                txn_dt, 
                                (case when wms_txn_amt_twd is null then 1 else wms_txn_amt_twd end) as txn_amt,
                                dta_src,
                                deduct_cnt,
                                etl_dt
                        from sinica.witwo103_hist 
                        where wm_txn_code='1'and txn_dt>='{d_start}' and txn_dt<='{d_end}' and deduct_cnt <=1), 
                cte2 as (select distinct replace(wm_prod_code, ' ', '') as wm_prod_code
                        from sinica.witwo106
                        where replace(prod_detail_type_code, ' ','') in ('FNDF','FNDD')) 
            select cte1.cust_no, cte1.wm_prod_code from cte1 inner join cte2 on cte1.wm_prod_code=cte2.wm_prod_code order by cust_no
            """.format(d_start=after_1d_dt, d_end=d_end)
        return pd.read_sql(sql, conn)  